# GENESIS' ORIGINAL CODE

# OBJECTIVE: (TO MY BEST UNDERSTANDING)
1. Build a Recommender System
- Recommender system for matching job descriptions to resumes
- recommdner system that suggests jobs to candidates based on resumes
- build recommender system for matching job descriptions for gov databases
2. Do something with government data
3. Build an app


Key Methods:
- embedding-based recommendations
- fine-tuned transformer model

## code reactions

- embedding models review needed
- review langchain embeddings library


# Dez Next Steps

1. Reproduce and run Genesis code with redacted pii data
2. Add additional LLMs
3. Figure Out How to make the prompt engineering run quicker, maybe do 1 resume at a time then slowly increase the dataset size
4. Figure Out How difficult is it to get api access to the suggested prompt engineering models

Scraped Resumes and Prompt Enegineering (GR Version)
###LLMs used: GPT4, Cohere and Llama 2
###Date: 10/30/2024
###Saxa 4#

**Data source:**  Used "modified_file 1.csv” which is the 2,483 resumes that was scraped online. This file was cleaned up into a “more_resume.json” fine. I used this file as a case study for prompt engineering.
o	Note: This code does NOT incorporate PII identification and redaction, which is Dezmond’s code.


**Key Highlights of	Prompt engineering steps:**

*Step 1.* --> Embedding: Used Hugging Face "all-mpnet-base-v2" as a pre-trained sentence transformer base model designed for generating sentence embeddings (numerical represetnations of sentences that capture semantics (i.e finance, HR, technology etc).

*Step 2* -->Prompt Engineering- started shell of code, but did not get code to run as step 1 above took 2+ hours and was still running.

*   Prompt Engineering 1: Open AI's GPT 4
      -  Updated code to call GR's specific API
      - Updated query prompts for resumes, but did not see output yet
*   Prompt Engineering 2:  Cohere - need to add in Specific API
*   Prompt Engineering 3:  Llama 2- Requested user access request to Meta via Hugging Face, awaiting access approval




***

In [ ]:

!pip install python-docx==1.1.0
!pip install pdfplumber==0.9.0

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

***

## Adding in the Other Resumes for Prompt Engineering exercise




In [ ]:
more_resumes = pd.read_csv('/content/drive/My Drive/Capstone(Saxa 4)/04Code/modified_file 1.csv')

In [ ]:
more_resumes.to_json('more_resumes.json', orient = 'records', lines = True)

In [ ]:
more_resumes.rename(columns = {'Resume_str': 'text'}, inplace = True)

In [ ]:
more_resumes

In [ ]:
more_resumes = pd.DataFrame(more_resumes)
more_resumes

In [ ]:
more_resumes.dtypes

In [ ]:
#more_resumes['text'] = more_resumes['text'].apply(lambda x: x.encode('utf-8').decode('utf-8'))

In [ ]:
more_resumes.dtypes

In [ ]:
#more_resumes.drop(columns = ['Resume_html'], inplace = True)

In [ ]:
more_resumes

In [ ]:
more_resumes.columns = more_resumes.columns.str.strip()

In [ ]:
more_resumes['text'] = more_resumes['text'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
more_resumes['text'] = more_resumes['text'].str.lower()

In [ ]:
more_resumes

In [ ]:
more_resumes['text'] = more_resumes['text'].str.replace('[^\w\s]', '', regex=True)

In [ ]:
cats_more_res = more_resumes['Category'].nunique()
cats_more_res

In [ ]:
cat_counts = more_resumes['Category'].value_counts()    #Look at CSV and get get rid of the 1's
cat_counts

In [ ]:
finance = more_resumes[more_resumes['Category'] == 'FINANCE']

In [ ]:
engineering = more_resumes[more_resumes['Category'] == 'ENGINEERING']

In [ ]:
more_resumes

In [ ]:
!pip install langchain==0.0.240 transformers sentence_transformers datasets faiss-cpu

### PROMPT ENGINEERING ##

In [ ]:
#import libraries

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document # Import the Document class
from datasets import load_dataset


In [ ]:
# 1. Load the Hugging Face Embeddings model
  ##Notes##
   # Code Purpose: loads a pre-trained sentence transformer model called "all-mpnet-base-v2" from Hugging Face.
   # Model insight: "all-mpnet-base-v2" is used to convert text (in this case, resume content) into numerical vectors called embeddings.
   # Functionality: Embeddings capture the semantic meaning of the text, allowing you to compare the similarity between different pieces of text based on their vector representations.
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

# 2. Load the more_resumes dataset
  # Assumption about dataset: Assuming 'more_resumes.json' is in the current directory
more_resumes = load_dataset('json', data_files='more_resumes.json')

# 3. Split resumes into smaller chunks for embedding
  #Code purpose: Divides each resume's text content into smaller chunks to handle potentially large documents.
  # Functionality: RecursiveCharacterTextSplitter: This object is used to split the text. It's configured to create
  # chunks of approximately 1000 characters with an overlap of 200 characters between chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = []
for resume in more_resumes['train']:
    texts = text_splitter.split_text(resume['text'])
    # Create Document objects instead of dictionaries
    docs.extend([Document(page_content=t, metadata={'source': resume['Category']}) for t in texts])

# 4. Embed the resume chunks and store them in a FAISS vectorstore
  #Purpose: Purpose: Creates a vector database using FAISS (Facebook AI Similarity Search) and
  # stores the embeddings of the resume chunks in it.
db = FAISS.from_documents(docs, embeddings)
db.save_local("more_resumes_faiss")

# Now the 'more_resumes' dataset is embedded and stored in a FAISS vectorstore.

In [ ]:
###Prompt Engineering 1: Open AI's GPT 4 ###

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os        #for API Key
import openai


# 1. Load the FAISS vectorstore
db = FAISS.load_local("more_resumes_faiss", embeddings)

# 2. Initialize OpenAI's GPT-4
# Make sure you set your OPENAI_API_KEY environment variable

llm = OpenAI(model_name="gpt-4", temperature=0)

# Set's GR's API key as an environment variable (recommended)
os.environ["genesis_gpt4_API_key"] = "genesis_gpt4_API_key" #Replace YOUR_API_KEY with your actual OpenAI key

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6a. Example usage 1
query = "Based on this resume, what are the top jobs I can qualify for?"
result = run_query(query)
print(result)

# 6b. Example usage 2
query1 = "I live in a high cost city and need to make at least $80k salary per year, based on my resume, what jobs should I apply for to get me this salary?"
result1 = run_query(query1)
print(result1)

In [ ]:
###Prompt Engineering 2:  Cohere ###

from langchain.chains import RetrievalQA
from langchain.llms import Cohere

# 1. Load the FAISS vectorstore
db = FAISS.load_local("more_resumes_faiss", embeddings)

# 2. Initialize Cohere
# Make sure you set your COHERE_API_KEY environment variable
llm = Cohere(model="command-xlarge-nightly", temperature=0)

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6. Example usage
query = "What are the skills of candidates in the Finance category?"
result = run_query(query)
print(result)

In [ ]:
###Prompt Engineering 3:  Llama 2 ###

from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp

# 1. Load the FAISS vectorstore
db = FAISS.load_local("more_resumes_faiss", embeddings)

# 2. Initialize Llama 2
# Replace with the path to your Llama 2 model file
llm = LlamaCpp(model_path="/path/to/your/llama-2-7b-chat.ggmlv3.q4_0.bin")


# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

# 4. Define your prompt template
prompt_template = """
Use the following pieces of context to answer the question at the end.
If you don't have enough information to answer the question, just say "I don't know".

Context:
{context}

Question:
{question}
"""

# 5. Function to run a query
def run_query(query):
    """Runs a query against the resume dataset using prompt engineering."""
    response = qa_chain.run(
        prompt_template.format(context=db.as_retriever().get_relevant_documents(query)[0].page_content, question=query)
    )
    return response #Corrected to return the response

# 6. Example usage
query = "What are the skills of candidates in the Finance category?"
result = run_query(query)
print(result)

# Testing Code with Redacted PII data

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
#from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
import os
import json
#from docx import Document
import pdfplumber
from bs4 import BeautifulSoup
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#spacy.cli.download("en_core_web_sm")
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
from plotly.offline import plot
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dezri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\dezri\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning:

`torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.



# Redacted with spacy

In [4]:
resumes = pd.read_csv('Resumes_with_PII_updated.csv')

In [5]:
#adjust this number to scale the size of the corpus
resumes = resumes.head(1)

In [6]:
resumes

,ID,Category,text
0,1,HR,Alexis Lee \n\nalexis.lee@gmail.com\n\n(728) 6...


In [7]:
nlp = spacy.load("en_core_web_lg")

In [8]:
def ner_pipeline(text):
    doc = nlp(text)
    entities = [{'word': ent.text, 'entity': ent.label_, 'score': ent.kb_id_} for ent in doc.ents]
    return entities

In [9]:
structured_documents_spacy_corpus = []

In [10]:
for index, row in resumes.iterrows():
    document_ID = row['ID']
    document_Category = row['Category']
    document_text = row['text']

    entities = ner_pipeline(document_text)

    document_data = {
        'ID': document_ID,
        'Category': document_Category,
        'text': document_text,
        'entities': entities  
    }

    structured_documents_spacy_corpus.append(document_data)

In [11]:
for doc in structured_documents_spacy_corpus:
    print(doc)

{'ID': '1', 'Category': 'HR', 'text': "Alexis Lee \n\nalexis.lee@gmail.com\n\n(728) 660-2201\n         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Com

In [12]:
print(f"Total documents in structured_documents: {len(structured_documents_spacy_corpus)}")
for doc in structured_documents_spacy_corpus:
    print(f"ID: {doc['ID']}, Entities Count: {len(doc['entities'])}")


Total documents in structured_documents: 1
ID: 1, Entities Count: 73


In [13]:
spacy_corpus_redacted_text = []

In [14]:
thresholds = {
    'PERSON': 0.75,
    'ORG': 0.99,
    'GPE': 0.99, 
}

In [15]:
for i, document in enumerate(structured_documents_spacy_corpus):
    original_text = document['text']
    redacted_text = original_text
    entities_sorted = sorted(document.get('entities', []), key=lambda x: len(x['word']), reverse=True)

    print(f"Processing Document {i + 1}: {document['ID']}")
    print(f"Entities from document {i + 1}:")
    
    for entity in entities_sorted:
        entity_text = entity['word']
        entity_label = entity['entity']
        score = entity.get('score', 1.0)
        
       #Makes sure score is treated as a float
    score = entity.get('score')
    if isinstance(score, str):
        try:
            score = float(score)
        except ValueError:
            score = 0.0  #default if conversion fails
    else:
        score = score if score is not None else 1.0  #fallback to 1.0 if score no score
 
    
        if entity_label in thresholds and score >= thresholds[entity_label]:
            print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.4f}")

    for entity in entities_sorted:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

    redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)
    redacted_text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[Redacted Phone]', redacted_text)
    redacted_text = re.sub(r'\b(?:https?://)?(?:www\.)?linkedin\.com/in/[a-zA-Z0-9._-]+\b', '[Redacted Website]', redacted_text)
    
    
    spacy_corpus_redacted_text.append({
        'ID': row['ID'],                #row instead of document to build seperate columns in output
        'Category': row['Category'],    
        'Redacted Text': redacted_text   
    })

#list to a df
spacy_redacted_resumes = pd.DataFrame(spacy_corpus_redacted_text)

#Save to CSV
spacy_redacted_resumes.to_csv('spacy_redacted_documents_with_id_and_category.csv', index=False)



Processing Document 1: 1
Entities from document 1:


In [16]:
print(f"Total number of redacted documents: {len(spacy_corpus_redacted_text)}")

Total number of redacted documents: 1


In [17]:
for i, text in enumerate(spacy_corpus_redacted_text):
    print(f"Redacted Text for Document {i + 1}:")
    print(text)

Redacted Text for Document 1:
{'ID': '1', 'Category': 'HR', 'Redacted Text': "[Redacted] \n\n[Redacted Email]\n\n([Redacted]) [Redacted]-2201\n         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated [Redacted] Manager with [Redacted] of experience in [Redacted].   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  [Redacted] management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           [Redacted][Redacted] DOT Supervisor Training Certification  Certified by [Redacted] in [Redacted]Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    [Redacted]  ,   [Redacted]   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars 

In [18]:
document_index = 0

In [19]:
if document_index < len(spacy_corpus_redacted_text):
    print(f"Redacted Text for Document 1:")
    print(spacy_corpus_redacted_text[document_index])
else:
    print("Document does not exist.")

Redacted Text for Document 1:
{'ID': '1', 'Category': 'HR', 'Redacted Text': "[Redacted] \n\n[Redacted Email]\n\n([Redacted]) [Redacted]-2201\n         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated [Redacted] Manager with [Redacted] of experience in [Redacted].   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  [Redacted] management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           [Redacted][Redacted] DOT Supervisor Training Certification  Certified by [Redacted] in [Redacted]Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    [Redacted]  ,   [Redacted]   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars 

# adding Genesis Code
- dataset csv name = spacy_redacted_documents_with_id_and_category.csv
https://python.langchain.com/docs/introduction/

In [ ]:
#pip install langchain

In [ ]:
#pip install langchain-core

In [2]:
#pip install langchain-community

  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/c8/4c/28602f96a6b5a2cad9ad474906a6eff0c65550bed844ce6c81478589a089/langchain_community-0.3.4-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/c3/be/d0d44e092656fe7a06b55e6103cbce807cdbdee17884a5367c68c9860853/dataclasses_json-0.6.7-py3-none-any.whl.metadata
  Obtaining dependency information for httpx-sse<0.5.0,>=0.4.0 from https://files.pythonhosted.org/packages/e1/9b/a181f281f65d776426002f330c31849b86b31fc9d848db62e16f03ff739f/httpx_sse-0.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic-settings<3.0.0,>=2.4.0 from https://files.pythonhosted.org/packages/34/19/26bb6bdb9fdad5f0dfce538780814084fb667b4bc37fcb28459c14b8d3b5/pydantic_settings-2.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for marshmallow<4.0.0,>=3.18.0 from https://files.pythonhosted.

In [ ]:
#pip install langchain-openai

In [ ]:
#pip install langchain-huggingface

In [34]:
#pip install --upgrade pydantic

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tf-keras

  Obtaining dependency information for tf-keras from https://files.pythonhosted.org/packages/8a/ed/e08afca471299b04a34cd548e64e89d0153eda0e6cf9b715356777e24774/tf_keras-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow<2.19,>=2.18 from https://files.pythonhosted.org/packages/cf/24/271e77c22724f370c24c705f394b8035b4d27e4c2c6339f3f45ab9b8258e/tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.18.0 from https://files.pythonhosted.org/packages/76/ad/fa6c508a15ff79cb5409294c293388e0999b7d480f84b65e4287277434fe/tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorboard<2.19,>=2.18 from https://files.pythonhosted.org/packages/b1/de/021c1d407befb505791764ad2cbd56ceaaa53a746baed01d2e2143f05f18/tensorboard-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for numpy<2.1.0,>=1.26.0 from https://files.pythonhosted.org/packages/eb/57/3a3f1

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\dezri\\anaconda3\\Lib\\site-packages\\~ensorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Consider using the `--user` option or check the permissions.



In [10]:
pip install langchain langchain-huggingface tf-keras

  Obtaining dependency information for tf-keras from https://files.pythonhosted.org/packages/8a/ed/e08afca471299b04a34cd548e64e89d0153eda0e6cf9b715356777e24774/tf_keras-2.18.0-py3-none-any.whl.metadata
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for numpy<2,>=1 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 330.3 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.0 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 466.1 kB/s eta 0:00:00
  Obtaining dependency information for tensorflow<2.19,>=2.18 from https://files.pythonhosted.org/packages/cf/24/271e77c22724f370c24c705

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.


In [13]:
pip install keras==2.11.0

  Obtaining dependency information for keras==2.11.0 from https://files.pythonhosted.org/packages/de/44/bf1b0eef5b13e6201aef076ff34b91bc40aace8591cd273c1c2a94a9cc00/keras-2.11.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 330.3 kB/s eta 0:00:06
   - -------------------------------------- 0.1/1.7 MB 469.7 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/1.7 MB 1.1 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.7 MB 1.6 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.7 MB 2.6 MB/s eta 0:00:01
   -------------------------- ------------- 1.1/1.7 MB 4.2 MB/s eta 0:00:01
   ---------------------------------- ----- 1.5/1.7 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires keras>=3.5.0, but you have keras 2.11.0 which is incompatible.


In [14]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from datasets import load_dataset
import keras

In [15]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

In [16]:

# 1. Load the Hugging Face Embeddings model
  ##Notes##
   # Code Purpose: loads a pre-trained sentence transformer model called "all-mpnet-base-v2" from Hugging Face.
   # Model insight: "all-mpnet-base-v2" is used to convert text (in this case, resume content) into numerical vectors called embeddings.
   # Functionality: Embeddings capture the semantic meaning of the text, allowing you to compare the similarity between different pieces of text based on their vector representations.
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

# 2. Load the more_resumes dataset
  # Assumption about dataset: Assuming 'more_resumes.json' is in the current directory
more_resumes = pd.read_csv('spacy_redacted_documents_with_id_and_category.csv')

# 3. Split resumes into smaller chunks for embedding
  #Code purpose: Divides each resume's text content into smaller chunks to handle potentially large documents.
  # Functionality: RecursiveCharacterTextSplitter: This object is used to split the text. It's configured to create
  # chunks of approximately 1000 characters with an overlap of 200 characters between chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = []
for resume in more_resumes['train']:
    texts = text_splitter.split_text(resume['text'])
    # Create Document objects instead of dictionaries
    docs.extend([Document(page_content=t, metadata={'source': resume['Category']}) for t in texts])

# 4. Embed the resume chunks and store them in a FAISS vectorstore
  #Purpose: Purpose: Creates a vector database using FAISS (Facebook AI Similarity Search) and
  # stores the embeddings of the resume chunks in it.
db = FAISS.from_documents(docs, embeddings)
db.save_local("more_resumes_faiss")

# Now the 'more_resumes' dataset is embedded and stored in a FAISS vectorstore.

RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.